# Eager and Lazy APIs

## Resumen del notebook

Este notebook compara y ejemplifica las dos APIs principales de Polars: la API eager (ejecución inmediata) y la API lazy (ejecución diferida y optimizable). Proporciona teoría, ejemplos prácticos reproducibles y pautas para elegir la API adecuada según el caso de uso.

### Objetivos
- Explicar las diferencias conceptuales entre Eager y Lazy.
- Mostrar ejemplos prácticos en Polars para cada API.
- Demostrar cómo la API lazy permite optimizaciones y mejor rendimiento en cargas grandes.
- Comparar tiempos y consumo de memoria entre workflows equivalentes.
- Visualizar resultados con Plotnine para apoyar el análisis.

### Estructura del notebook
1. Introducción y motivación: cuándo usar cada API.
2. Preparación: carga de datos desde el directorio `data/` (ver Cap. 2).
3. Sección Eager:
    - Operaciones básicas (selección, filtrado, agregación).
    - Transformaciones elementales y uso interactivo.
4. Sección Lazy:
    - Construcción de planos de ejecución (lazy frames).
    - Optimización automática y ejecución con `collect()`.
    - Ejemplos de pushdown, projection pruning y fusión de expresiones.
5. Benchmarking:
    - Comparativa de latencia y uso de memoria en tareas representativas.
6. Visualización y análisis:
    - Gráficos con Plotnine para resultados agregados y comparaciones.
7. Conclusiones y recomendaciones prácticas.

### Notas y buenas prácticas
- Los datos deben encontrarse en el subdirectorio `data/` (no incluir archivos de datos en el repositorio).
- Usaremos Polars para manipulación de datos y Plotnine para visualización.
- Mantener el entorno virtual fuera del control de versiones.
- Incluir ejemplos reproducibles y explicaciones claras para facilitar la experimentación.

In [1]:
import polars as pl

## Eager API: DataFrame

In [2]:
%%time
trips = pl.read_parquet("data/taxi/yellow_tripdata_*.parquet")
sum_per_vendor = trips.group_by("VendorID").sum()

income_per_distance_per_vendor = sum_per_vendor.select(
    "VendorID",
    income_per_distance=pl.col("total_amount") / pl.col("trip_distance"),
)

top_three = income_per_distance_per_vendor.sort(
    by="income_per_distance", descending=True
).head(3)

print(top_three)

shape: (3, 2)
┌──────────┬─────────────────────┐
│ VendorID ┆ income_per_distance │
│ ---      ┆ ---                 │
│ i64      ┆ f64                 │
╞══════════╪═════════════════════╡
│ 1        ┆ 6.434789            │
│ 6        ┆ 5.296493            │
│ 5        ┆ 4.731557            │
└──────────┴─────────────────────┘
CPU times: total: 24.4 s
Wall time: 2.87 s


In [3]:
names_lf = pl.LazyFrame(
    {"name": ["Alice", "Bob", "Charlie"], "age": [25, 30, 35]}
)

erroneous_query = names_lf.with_columns(
    sliced_age=pl.col("age").slice(1, 3)
)

result_df = erroneous_query.collect()

ShapeError: unable to add a column of length 2 to a DataFrame of height 3

In [4]:
%%time
trips = pl.scan_parquet("data/taxi/yellow_tripdata_*.parquet")
sum_per_vendor = trips.group_by("VendorID").sum()

income_per_distance_per_vendor = sum_per_vendor.select(
    "VendorID",
    income_per_distance=pl.col("total_amount") / pl.col("trip_distance"),
)

top_three = income_per_distance_per_vendor.sort(
    by="income_per_distance", descending=True
).head(3)

print(top_three.collect())

shape: (3, 2)
┌──────────┬─────────────────────┐
│ VendorID ┆ income_per_distance │
│ ---      ┆ ---                 │
│ i64      ┆ f64                 │
╞══════════╪═════════════════════╡
│ 1        ┆ 6.434789            │
│ 6        ┆ 5.296493            │
│ 5        ┆ 4.731557            │
└──────────┴─────────────────────┘
CPU times: total: 2.7 s
Wall time: 515 ms


In [5]:
lf = pl.LazyFrame({"col1": [1, 2, 3], "col2": [4, 5, 6]})
# ... Some heavy computation ...
print(lf.collect())
print(lf.with_columns(pl.col("col1") + 1).collect())

shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ i64  │
╞══════╪══════╡
│ 1    ┆ 4    │
│ 2    ┆ 5    │
│ 3    ┆ 6    │
└──────┴──────┘
shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ i64  │
╞══════╪══════╡
│ 2    ┆ 4    │
│ 3    ┆ 5    │
│ 4    ┆ 6    │
└──────┴──────┘


### Evitar recalculaciones en LazyFrame — resumen y explicación

Problema
- Cada llamada a `collect()` ejecuta el plan perezoso desde el principio. Si el LazyFrame representa cálculos costosos, múltiples `collect()` repiten todo el trabajo y aumentan tiempo/CPU.

## Functionality Differences

De los atributos que tiene un DataFrame, un LazyFrame no dispone de los que requieren que los datos estén materializados: `shape`, `height` y `flags`. El resto de atributos como `columns`, `dtypes`, `schema` y `width` sí están disponibles en un LazyFrame.

| Atributo  | DataFrame | LazyFrame |
|---|:--:|:--:|
| .columns | ✓ | ✓ |
| .dtypes  | ✓ | ✓ |
| .flags   | ✓ |  |
| .height  | ✓ |  |
| .schema  | ✓ | ✓ |
| .shape   | ✓ |  |
| .width   | ✓ | ✓ |

Notas breves:
- `shape`/`height` (número de filas) y `flags` solo se obtienen una vez que los datos están materializados; llama a `.collect()` y luego usa `df.shape`, `len(df)` o `df.flags`.
- Para inspecciones ligeras sin materializar todo, usa `.schema`, `.columns` o inspecciona el plan lazy con métodos de diagnóstico antes de `collect()`.




### Resumen — diferencias clave entre DataFrame (eager) y LazyFrame (lazy)

- Propósito
    - DataFrame: API eager, datos materializados; permite inspección, exportación y operaciones que requieren conocer los valores fila a fila.
    - LazyFrame: API lazy, construye planes de ejecución optimizables; las operaciones se añaden al plan y se ejecutan al llamar a `.collect()`.

### Agregaciones
- Verticales (p. ej. `.sum()`, `.max()`, `.mean()`, `.quantile()`) funcionan en DataFrame y LazyFrame.
- Horizontales (terminan en `_horizontal`, p. ej. `.sum_horizontal()`, `.mean_horizontal()`) operan fila a fila y requieren conocer la estructura/valores: solo están disponibles en DataFrames.
- Método `df.product()`: solo DataFrame.

### Métodos de cómputo fila‑a‑fila
- Solo en DataFrame: `df.fold()` (reducción personalizada entre columnas) y `df.hash_rows()` (hash por fila → UInt64).
- LazyFrame no dispone de estos métodos.

### Métodos descriptivos e inspección
- DataFrame tiene muchas utilidades: `df.describe()`, `df.estimated_size()`, `df.approx_n_unique()`, `df.n_unique()`, `df.is_unique()`, `df.is_empty()`, `df.null_count()`, etc.
- LazyFrame ofrece herramientas de diagnóstico del plan: `lf.explain()` y `lf.show_graph()`. Técnicamente `lf.describe()` existe pero materializa (collect) los datos.

### GroupBy
- La mayoría de métodos de `GroupBy` son iguales en ambas APIs (`.agg()`, `.sum()`, `.mean()`, `.max()`, `.min()`, `.n_unique()`, `.quantile()`, `.map_groups()`, etc.).
- Diferencia: DataFrame permite iterar explícitamente sobre los grupos (`.__iter__()`), LazyFrame no.

### Exportación
- DataFrame: múltiples métodos de exportación (`.to_arrow()`, `.to_pandas()`, `.to_numpy()`, `.to_dicts()`, `.to_torch()`, `.to_struct()`, etc.).
- LazyFrame: no exporta directamente porque no tiene datos materializados; puede serializar planes y tiene métodos de "sink" para escribir a disco (ver streaming/sinks).

### Manipulación y selección
- Muchas operaciones comunes están disponibles en ambas APIs (`.filter()`, `.select()`, `.with_columns()`, `.join()`, `.group_by()`, `.sort()`, `.slice()`, `.unique()`, etc.).
- Algunas operaciones intrínsecamente fila‑dependientes o iterativas (p. ej. `.iter_rows()`, `.map_rows()`) son propias del DataFrame.

### Modo streaming / out‑of‑core (experimental)
- LazyFrame puede ejecutar en modo streaming para procesar datos que no caben en RAM:
    - `lf.collect(streaming=True)` — ejecuta en chunks y requiere que el resultado final quepa en RAM.
    - Sinks para escribir directamente: `lf.sink_csv()`, `lf.sink_ipc()`, `lf.sink_parquet()`, `lf.sink_ndjson()`.
- Útil para mover el límite de procesamiento de la RAM al disco; API experimental.

### Recomendaciones prácticas
- Usar LazyFrame cuando se necesite optimización del plan, fusiones de expresiones y procesamiento de grandes volúmenes antes de materializar.
- Evitar múltiples `collect()` sobre el mismo `LazyFrame` para impedir recomputaciones; cachear o materializar intermedios si es necesario.
- Usar DataFrame cuando se necesiten operaciones fila‑a‑fila (horizontal), exportaciones, iteración por filas o inspecciones detalladas.
- Para workflows reproducibles y escalables, documentar uso de sinks/streaming y preferir operaciones que el motor lazy pueda optimizar.

Fin.

### Joining  a DataFrame with a LazyFrame

In [6]:
big_sales_data = pl.LazyFrame(
    {"sale_id": [101, 102, 103], "amount": [250, 150, 300]}
)
sales_metadata = pl.DataFrame(
    {"sale_id": [101, 102, 103], "category": ["A", "B", "A"]}
)

big_sales_data.join(sales_metadata, on="sale_id").collect()

TypeError: expected `other` join table to be a LazyFrame, not a 'DataFrame'

In [7]:
big_sales_data = pl.LazyFrame(
    {"sale_id": [101, 102, 103], "amount": [250, 150, 300]}
)
sales_metadata = pl.DataFrame(
    {"sale_id": [101, 102, 103], "category": ["A", "B", "A"]}
)

big_sales_data.join(sales_metadata.lazy(), on="sale_id").collect()

sale_id,amount,category
i64,i64,str
101,250,"""A"""
102,150,"""B"""
103,300,"""A"""


### Caching Intermittent Results

#### Explicación (resumida y concisa)

- Se define `lf` como un `LazyFrame` inicial (no se ejecutan cómputos aún).
- Comentario `# ... Some heavy computation ...` indica que antes podría haberse añadido un plan costoso al `LazyFrame`.
- `lf = lf.collect().lazy()`:
    - `collect()` materializa el plan perezoso y ejecuta los cómputos, devolviendo un `DataFrame` en memoria.
    - `.lazy()` convierte ese `DataFrame` de nuevo a un `LazyFrame`. Esto actúa como una forma explícita de cachear/guardar el resultado intermedio para evitar recomputaciones del plan original.
- `print(lf.collect())` muestra el `DataFrame` resultante ya materializado.
- `print(lf.with_columns(pl.col("col1") + 1).collect())` añade una nueva columna (col1+1) al plan perezoso basado en el resultado ya materializado y muestra el nuevo `DataFrame`.

Nota sobre rendimiento: materializar con `collect()` evita repetir cálculos costosos en llamadas posteriores, pero consume memoria (el resultado intermedio debe caber en RAM).

In [8]:
lf = pl.LazyFrame({"col1": [1, 2, 3], "col2": [4, 5, 6]})
# ... Some heavy computation ...
lf = lf.collect().lazy()  
print(lf.collect())
print(lf.with_columns(pl.col("col1") + 1).collect()) 

shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ i64  │
╞══════╪══════╡
│ 1    ┆ 4    │
│ 2    ┆ 5    │
│ 3    ┆ 6    │
└──────┴──────┘
shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ i64  │
╞══════╪══════╡
│ 2    ┆ 4    │
│ 3    ┆ 5    │
│ 4    ┆ 6    │
└──────┴──────┘
